# Assignment #3: Genetic Algorithm (Training Cars with GA)

### Required Libraries

In [ ]:
pip install pygame

In [1]:
import pygame
import time
import math
import random


pygame 2.5.2 (SDL 2.28.3, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Helping functions

In [2]:
def scale_image(img, factor):
    size = round(img.get_width() * factor), round(img.get_height() * factor)
    return pygame.transform.scale(img, size)

def blit_rotate_center(win, image, top_left, angle):
    rotated_image = pygame.transform.rotate(image, angle)
    new_rect = rotated_image.get_rect(
        center=image.get_rect(topleft=top_left).center)
    win.blit(rotated_image, new_rect.topleft)
    
def blit_text_center(win, font, text):
    render = font.render(text, 1, (200, 200, 200))
    win.blit(render, (win.get_width()/2 - render.get_width() /
                      2, win.get_height()/2 - render.get_height()/2))

### Abstract Classes

In [3]:
class GameInfo:
    LEVELS = 1

    def __init__(self, level=1):
        self.level = level
        self.started = False
        self.level_start_time = 0

    def next_level(self):
        self.level += 1
        self.started = False

    def reset(self):
        self.level = 1
        self.started = False
        self.level_start_time = 0

    def game_finished(self):
        return self.level > self.LEVELS

    def start_level(self):
        self.started = True
        self.level_start_time = time.time()

    def get_level_time(self):
        if not self.started:
            return 0
        return round(time.time() - self.level_start_time)

### Sensor Class

In [4]:
class SensorBullet:
    def __init__(self, car, base_angle, vel, color):
        self.x = car.x + CAR_WIDTH/2
        self.y = car.y + CAR_HEIGHT/2
        self.angle = car.angle
        self.base_angle = base_angle
        self.vel = vel
        self.color = color
        self.img = pygame.Surface((4, 4))
        self.fired = False
        self.hit = False
        self.last_poi = None

    def draw(self, win):
        pygame.draw.circle(win, self.color, (self.x, self.y), 2)

    def fire(self, car):
        self.angle = car.angle + self.base_angle
        self.x = car.x + CAR_WIDTH/2
        self.y = car.y + CAR_HEIGHT/2
        self.fired = True
        self.hit = False

    def move(self):
        if(self.fired):
            radians = math.radians(self.angle)
            vertical = math.cos(radians) * self.vel
            horizontal = math.sin(radians) * self.vel

            self.y -= vertical
            self.x -= horizontal

    def collide(self, x=0, y=0):
        bullet_mask = pygame.mask.from_surface(self.img)
        offset = (int(self.x - x), int(self.y - y))
        poi = TRACK_BORDER_MASK.overlap(bullet_mask, offset)
        if poi:
            self.fired = False
            self.hit = True
            self.last_poi = poi
        return poi

    def draw_line(self, win, car):
        if self.hit:
            pygame.draw.line(win, self.color, (car.x + CAR_WIDTH/2, car.y + CAR_HEIGHT/2), (self.x, self.y), 1)
            pygame.display.update()

    def get_distance_from_poi(self, car):
        if self.last_poi is None:
            return -1
        return math.sqrt((car.x - self.last_poi[0])**2 + (car.y - self.last_poi[1])**2)

### Drawing and Collision functions

In [5]:
def draw(win, images, cars, level_time, checkpoints, n_cars, generation, most_checkpoints, max_for_this_gen):
        
    for img, pos in images:
        win.blit(img, pos)
        
    for checkpoint in checkpoints:
        win.blit(checkpoint['image'], checkpoint['position'])
        
    win.blit(TRACK_BORDER, (0, 0))


    time_text = MAIN_FONT.render(
        f"Time: {level_time}s", 1, (255, 255, 0))
    win.blit(time_text, (10, HEIGHT - time_text.get_height() - 40))
    
    c_text = MAIN_FONT.render(
        f"No. of Cars: {n_cars}", 1, (255, 255, 0))
    win.blit(c_text, (10, HEIGHT - time_text.get_height() - 80))
    
    g_text = MAIN_FONT.render(
        f"Generation: {generation}", 1, (255, 255, 0))
    win.blit(g_text, (10, HEIGHT - time_text.get_height() - 120))
    
    m_text = MAIN_FONT.render(
        f"Max Checkpoints: {most_checkpoints}", 1, (255, 255, 0))
    win.blit(m_text, (10, HEIGHT - time_text.get_height() - 160))
    
    l_text = MAIN_FONT.render(
        f"Max for this generation: {max_for_this_gen}", 1, (255, 255, 0))
    win.blit(l_text, (10, HEIGHT - time_text.get_height() - 200))



    for car in cars:
        if car.active:
            car.draw(win)
            for bullet in car.sensors:
                bullet.draw(win)
        

    pygame.display.update()


def handle_collision(car, checkpoints:list(),FINISH_MASK):
    if car.collide(TRACK_BORDER_MASK) != None:
        car.bounce()
        car.active = False

    for bullet in car.sensors:
        if bullet.collide() != None:
            bullet.draw_line(WIN, car)

    player_finish_poi_collide = car.collide(
        FINISH_MASK, *FINISH_POSITION)
    if player_finish_poi_collide != None:
        if player_finish_poi_collide[1] == 0:
            car.bounce()
            car.score -= 0.01
            car.active = False
        else:
            car.completed = True
            return True
            
    idx = car.checkpoints
    if idx < 30:
        player_checkpoint_collide = car.collide(checkpoints[idx]['mask'], *checkpoints[idx]['position'])
        if player_checkpoint_collide != None:
            car.checkpoints += 1
            car.score += 1
            car.time = game_info.get_level_time()
    else:
        car.completed = True
        return True

### Abstract Car Class

In [6]:
class AbstractCar:
    def __init__(self):
        self.img = self.IMG
        self.vel = 0
        self.angle = 0
        self.x, self.y = self.START_POS

    def draw(self, win):
        blit_rotate_center(win, self.img, (self.x, self.y), self.angle)
        
    def collide(self, mask, x=0, y=0):
        car_mask = pygame.mask.from_surface(self.img)
        offset = (int(self.x - x), int(self.y - y))
        poi = mask.overlap(car_mask, offset)
        return poi

    def action(self,direction, acceleration):
        if direction == 1:
            self.vel = acceleration
        elif direction == 2:
            self.vel = -acceleration / 2
        elif direction == 3:
            self.angle += -acceleration
        elif direction == 4:
            self.angle += acceleration
            
        self.move()

    def move(self):
        radians = math.radians(self.angle)
        vertical = math.cos(radians) * self.vel
        horizontal = math.sin(radians) * self.vel 

        self.y -= vertical
        self.x -= horizontal
        

### Window Initialization

In [7]:
pygame.font.init()

WIDTH, HEIGHT = 800,800
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Genetic Algorithm on cars! (Assignment #3)")

MAIN_FONT = pygame.font.SysFont("Times New Roman", 44)

GRASS = scale_image(pygame.image.load("imgs/grass.jpg"), 2.5)
TRACK = scale_image(pygame.image.load("imgs/track.png"), 0.9)

TRACK_BORDER = scale_image(pygame.image.load("imgs/track-border.png"), 0.9)
TRACK_BORDER_MASK = pygame.mask.from_surface(TRACK_BORDER)

FINISH = pygame.image.load("imgs/finish.png")

FINISH_MASK = pygame.mask.from_surface(FINISH)
FINISH_POSITION = (130, 250)


RED_CAR = scale_image(pygame.image.load("imgs/red-car.png"), 0.40)

CAR_WIDTH, CAR_HEIGHT = RED_CAR.get_width(), RED_CAR.get_height()

### Agent Car Class

In [8]:
RED_CAR = scale_image(pygame.image.load("imgs/red-car.png"), 0.40)

CAR_WIDTH, CAR_HEIGHT = RED_CAR.get_width(), RED_CAR.get_height()
class AgentCar(AbstractCar):
    IMG = RED_CAR
    START_POS = (160, 200)

    def __init__(self):
        super().__init__()
        self.sensors = [SensorBullet(self, 45, 20, (0, 0, 255)), SensorBullet(self, 25, 20, (0, 0, 255)), SensorBullet(self, 0, 20, (0, 255, 0)), SensorBullet(self, -25, 20, (0, 0, 255)), SensorBullet(self, -45, 20, (0, 0, 255))]
        self.completed = False
        self.active = True
        self.checkpoints = 0
        self.score = 0
        self.time = 0
        self.sensor_combinations = []
        for i in range(2**5):
            binary_str = bin(i)[2:].zfill(5)
            self.sensor_combinations.append([int(bit) for bit in binary_str])
        self.chromosome = self.initailze_chromosome()
    
    def reduce_speed(self):
        self.vel = max(self.vel - 0.1 / 2, 0)
        self.move()

    def bounce(self):
        self.vel = -self.vel
        self.move()

    def fireSensors(self): 
        for bullet in self.sensors:
            bullet.fire(self)
    
    def sensorControl(self):
        #print(contains(self.sensors, lambda x: x.hit))

        for bullet in self.sensors:
            if not bullet.fired:
                bullet.fire(self)

        for bullet in self.sensors:
            bullet.move()
    
    def get_distance_array(self):
        continous = [bullet.get_distance_from_poi(self) for bullet in self.sensors]
        discrete_distance = [1 if i <= 75 else 0 for i in continous]
        return discrete_distance
    
    def reset(self):
        self.x, self.y = self.START_POS
        self.angle = 0
        self.vel = 0
        self.checkpoints = 0
        self.score = 0
        self.time = 0
        self.active = True
        
    def reset_score(self):
        self.score = 0
        
    def initailze_chromosome(self):
        chromosomes = {}

        for state in self.sensor_combinations:
            direction = random.randint(1, 4)
#             acc = random.randint(1, 10)
            chromosomes[str(state)] = direction

        return chromosomes
    
    def move_car(self):
        input_state = self.get_distance_array()
        
        actions = self.chromosome[str(input_state)]
        
        speed = 5
        self.action(actions, speed)

### Fitness, Crossover, Mutation Functions

In [9]:
def calculate_fitness(car):
    # Set initial fitness to a low value
    fitness = 100  # Base value
    checkpoint_bonus = 100 * car.checkpoints
    time_penalty = 1 / (car.time + 1)  # Inverse of time (higher is better)
    completion_reward = 0

    # If the car has completed the track, reward completion
    if car.completed:
        completion_reward = 1000  # Reward for completion
    else:
        # Penalize cars that do not complete the track
        fitness -= 10  # Small penalty for non-completion

    fitness += completion_reward + time_penalty + checkpoint_bonus

    return fitness


# ----------------------------------------------------------------------#
#               Your crossover and mutation functions here              #
#-----------------------------------------------------------------------#


def crossover(parent1, parent2):
    child1 = {}
    child2 = {}

    # pick first half of parent1 and second half of parent2 for child 1
    for key in parent1.keys():
        if random.random() < 0.5:
            child1[key] = parent1[key]
        else:
            child1[key] = parent2[key]

    # pick first half of parent2 and second half of parent1 for child 2
    for key in parent2.keys():
        if random.random() < 0.5:
            child2[key] = parent2[key]
        else:
            child2[key] = parent1[key]

    print("Crossover of parents: ", parent1, " and ", parent2, " produced children: ", child1, " and ", child2 , "\n")
    
    return child1, child2



def mutation(chromosome, mutationRate):
    for key in chromosome.keys():
        if random.random() < mutationRate:
            chromosome[key] = random.randint(1, 4)
    
    return chromosome


def select_parent(cars):
    fitness_values = [calculate_fitness(car) for car in cars]
    # Calculate the sum of all fitness values
    total_fitness = sum(fitness_values)

    # Generate a random value between 0 and the total fitness
    random_value = random.uniform(0, total_fitness)

    # Initialize variables to keep track of the selected car and its fitness
    selected_car = None
    current_fitness = 0

    # Iterate through the cars and select the one that corresponds to the random value
    for car, fitness in zip(cars, fitness_values):
        current_fitness += fitness
        if current_fitness >= random_value:
            selected_car = car
            break

    return selected_car


# Driver Code/ Training

In [ ]:
#Intialize population
pop_size = 100
cars  = [AgentCar() for _ in range(pop_size)]
active_cars = pop_size
generation = 0
max_checkpoints = 0

highest_checkpoint_num = 0


while True:
    active_cars = pop_size
    pygame.font.init()
    
    MAIN_FONT = pygame.font.SysFont("Times New Roman", 44)
    GRASS = scale_image(pygame.image.load("imgs/grass.jpg"), 2.5)
    TRACK = scale_image(pygame.image.load("imgs/track.png"), 0.9)
    TRACK_BORDER = scale_image(pygame.image.load("imgs/track-border.png"), 0.9)
    TRACK_BORDER_MASK = pygame.mask.from_surface(TRACK_BORDER)
    FINISH = pygame.image.load("imgs/finish.png")
    FINISH_MASK = pygame.mask.from_surface(FINISH)
    FINISH_POSITION = (130, 250)
    RED_CAR = scale_image(pygame.image.load("imgs/red-car.png"), 0.40)
    CAR_WIDTH, CAR_HEIGHT = RED_CAR.get_width(), RED_CAR.get_height()

    checkpoint_image = pygame.image.load("imgs/finish.png")
    checks = [
            (130, 150),(130, 60), (110,30), (30,60) ,(0,150), (0, 250) ,(0,350),(90,550),(190,650),(350,650), #default checkpoints
            (350,550), (400, 450), (500, 430),(530, 450), (550,550), (550,650), (700,650),(700,550), (700,450),(680,350), (630,320), (630, 220), (630,30), (550,30), (450,30), (350,30),
            (700, 150),(230, 150),(230, 250),
            (550,320),(550,200),(655,680)
        ]
    checkpoints = list()

    for position in checks:
        if position in [(550,320),(550,200),(655,680),(110,30),(500, 430),(630,320) ,(630, 220),(630,30), (550,30), (450,30), (350,30)]:
            checkpoint_mask = pygame.mask.from_surface(pygame.transform.rotate(checkpoint_image, 90))
            checkpoints.append({
                'image': pygame.transform.rotate(checkpoint_image, 90),
                'mask': checkpoint_mask,
                'position': position
            })
        elif position in [(130, 60),(530, 450),(680,350)]:
            checkpoint_mask = pygame.mask.from_surface(pygame.transform.rotate(checkpoint_image, 45))
            checkpoints.append({
                'image': pygame.transform.rotate(checkpoint_image, 45),
                'mask': checkpoint_mask,
                'position': position
            })

        elif position in [(30, 60),(400, 450)]:
            checkpoint_mask = pygame.mask.from_surface(pygame.transform.rotate(checkpoint_image, 135))
            checkpoints.append({
                'image': pygame.transform.rotate(checkpoint_image, 135),
                'mask': checkpoint_mask,
                'position': position
            })
        else:
            checkpoint_mask = pygame.mask.from_surface(checkpoint_image)
            checkpoints.append({
                'image': checkpoint_image,
                'mask': checkpoint_mask,
                'position': position
            })
    clock = pygame.time.Clock()
    images = [(GRASS, (0, 0)), (TRACK, (0, 0)), (FINISH, FINISH_POSITION)]
    
    WIDTH, HEIGHT = 800,800
    WIN = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Autonomus Car2!")
        
    generation += 1
    print("Generation: ", generation)
    print("\n\n\n\n")
    game_info = GameInfo()
    prev_parents = list()
    prev_cum_sum = 0
    
    factor = 3
    max_time = factor *  max_checkpoints + 10
    check = False

    max_for_this_gen = 0

    temp_cars = cars.copy()
    temp_cars.sort(key=calculate_fitness, reverse=True)

    while game_info.get_level_time() < max_time and active_cars > 0:
        temp_cars.sort(key=calculate_fitness, reverse=True)

    #     clock.tick(FPS)
        if game_info.started == False:
            game_info.start_level()

        max_for_this_gen = temp_cars[0].checkpoints
        draw(WIN, images, cars, max_time - game_info.get_level_time() ,checkpoints,active_cars, generation, highest_checkpoint_num, max_for_this_gen)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                break

        active_cars = 0
        for car in cars:
            if car.active and not car.completed:
                active_cars+=1
                car.sensorControl()
                car.move_car()
                check = handle_collision(car, checkpoints, FINISH_MASK)
                highest_checkpoint_num = max(highest_checkpoint_num, car.checkpoints)
                print("Car: ", cars.index(car), "Checkpoints: ", car.checkpoints, "Score: ", car.score, "Fitness: ", calculate_fitness(car))

            
    
    # ----------------------------------------------------------------------#
    #                   Your GA code stars below this point                 #
    #-----------------------------------------------------------------------#
    
    # Here are the variables you will be working with:
    
    # pop_size: 100
    # # cars: which contain list of cars (all of them instances of AgentCar())
    # cars = [AgentCar() for _ in range(pop_size)]
    # <ANYCAR>.chromosome: chromosome of the car in the form [Sensor Values]:Movement as a dictionary
    # <ANYCAR>.completed which upon True, will be the end goal of the GA
    # calculate_fitness(<ANYCAR>): returns fitness value of the car, higher is better
    
    # Here is the general algorithm we recommend you to follow. 
    # If you want to change the algorithm a bit, feel free.
    
    # Sort cars list by fitness value (this is already done for you below)
    cars.sort(key=calculate_fitness, reverse=True)

    num_elite = int(0.35 * pop_size)
    elite = cars[:num_elite]
    new_generation = []
    # print fitnesses of cars with the number of checkpoints reached

    
    # Check if best car is has reached all checkpoints (<ANYCAR>.completed)
    if cars[0].completed:
        print("\n\n\nBest car has reached all checkpoints in ",   generation, " generations!\n\n\n" )
    #     if true, then just escape the loop using a break statement
        break
    # Iterate from 0 to half of the population value
    for i in range(pop_size - num_elite):
        parent1 = select_parent(cars)
        parent2 = select_parent(cars)

        
        child1 = AgentCar()
        child2 = AgentCar()

        child1.chromosome, child2.chromosome = crossover(parent1.chromosome, parent2.chromosome)
        
        child1.chromosome = mutation(child1.chromosome, 0.18)
        child2.chromosome = mutation(child2.chromosome, 0.18)

        new_generation.append(child1)
        new_generation.append(child2)
    # Replace cars list with new generation list
    cars = elite + new_generation

    try:
        max_checkpoints = cars[0].checkpoints
    except IndexError:
        max_checkpoints = 0
    
    # If you perform any sort of elitism or carry over old parents into the new generation, 

    # make sure to reset the state of each car in the old generation. 

    # The command for that is <ANYCAR>.reset(). This command will not return any value.
    for car in cars:
        car.reset()
        
     # Good luck!

    
    # This line of code self-adjusts the time for each generation, please do not remove it :)
    

    # clear_output()
    
    # Let this stay here, it will quit the game at each generation and begin a new one
    pygame.quit()

# clear_output()
pygame.quit()    

# Saving chromosomes of population which cleared 10 checkpoints

In [10]:
import pickle
file_name = "all_points.pkl"

# car_chromosomes = [car.chromosome for car in cars]

# print("Champion chromosomes:")
# print(car_chromosomes)

# open_file = open(file_name, "wb")
# pickle.dump(car_chromosomes, open_file)
# open_file.close()

open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

new_cars = list()
for i in range(len(loaded_list)):
    car = AgentCar()
    car.chromosome = loaded_list[i]
    new_cars.append(car)
new_cars

In [11]:
loaded_list[10]

{'[0, 0, 0, 0, 0]': 1,
 '[0, 0, 0, 0, 1]': 4,
 '[0, 0, 0, 1, 0]': 1,
 '[0, 0, 0, 1, 1]': 1,
 '[0, 0, 1, 0, 0]': 3,
 '[0, 0, 1, 0, 1]': 2,
 '[0, 0, 1, 1, 0]': 3,
 '[0, 0, 1, 1, 1]': 4,
 '[0, 1, 0, 0, 0]': 1,
 '[0, 1, 0, 0, 1]': 2,
 '[0, 1, 0, 1, 0]': 2,
 '[0, 1, 0, 1, 1]': 4,
 '[0, 1, 1, 0, 0]': 4,
 '[0, 1, 1, 0, 1]': 3,
 '[0, 1, 1, 1, 0]': 1,
 '[0, 1, 1, 1, 1]': 2,
 '[1, 0, 0, 0, 0]': 1,
 '[1, 0, 0, 0, 1]': 1,
 '[1, 0, 0, 1, 0]': 4,
 '[1, 0, 0, 1, 1]': 1,
 '[1, 0, 1, 0, 0]': 3,
 '[1, 0, 1, 0, 1]': 2,
 '[1, 0, 1, 1, 0]': 1,
 '[1, 0, 1, 1, 1]': 2,
 '[1, 1, 0, 0, 0]': 1,
 '[1, 1, 0, 0, 1]': 1,
 '[1, 1, 0, 1, 0]': 3,
 '[1, 1, 0, 1, 1]': 1,
 '[1, 1, 1, 0, 0]': 1,
 '[1, 1, 1, 0, 1]': 1,
 '[1, 1, 1, 1, 0]': 1,
 '[1, 1, 1, 1, 1]': 1}

# Testing

In [12]:
pygame.font.init()

MAIN_FONT = pygame.font.SysFont("Times New Roman", 44)
GRASS = scale_image(pygame.image.load("imgs/grass.jpg"), 2.5)
TRACK = scale_image(pygame.image.load("imgs/track.png"), 0.9)
TRACK_BORDER = scale_image(pygame.image.load("imgs/track-border.png"), 0.9)
TRACK_BORDER_MASK = pygame.mask.from_surface(TRACK_BORDER)
FINISH = pygame.image.load("imgs/finish.png")
FINISH_MASK = pygame.mask.from_surface(FINISH)
FINISH_POSITION = (130, 250)
RED_CAR = scale_image(pygame.image.load("imgs/red-car.png"), 0.40)
CAR_WIDTH, CAR_HEIGHT = RED_CAR.get_width(), RED_CAR.get_height()

checkpoint_image = pygame.image.load("imgs/finish.png")
checks = [
            (130, 150),(130, 60), (110,30), (30,60) ,(0,150), (0, 250) ,(0,350),(90,550),(190,650),(350,650), #default checkpoints
            (350,550), (400, 450), (500, 430),(530, 450), (550,550), (550,650), (700,650),(700,550), (700,450),(680,350), (630,320), (630, 220), (630,30), (550,30), (450,30), (350,30),
            (700, 150),(230, 150),(230, 250),
            (550,320),(550,200),(655,680)
        ]
checkpoints = list()

highest_checkpoint_num = 0

for position in checks:
    if position in [(550,320),(550,200),(655,680),(110,30),(500, 430),(630,320) ,(630, 220),(630,30), (550,30), (450,30), (350,30)]:
        checkpoint_mask = pygame.mask.from_surface(pygame.transform.rotate(checkpoint_image, 90))
        checkpoints.append({
            'image': pygame.transform.rotate(checkpoint_image, 90),
            'mask': checkpoint_mask,
            'position': position
        })
    elif position in [(130, 60),(530, 450),(680,350)]:
        checkpoint_mask = pygame.mask.from_surface(pygame.transform.rotate(checkpoint_image, 45))
        checkpoints.append({
            'image': pygame.transform.rotate(checkpoint_image, 45),
            'mask': checkpoint_mask,
            'position': position
        })

    elif position in [(30, 60),(400, 450)]:
        checkpoint_mask = pygame.mask.from_surface(pygame.transform.rotate(checkpoint_image, 135))
        checkpoints.append({
            'image': pygame.transform.rotate(checkpoint_image, 135),
            'mask': checkpoint_mask,
            'position': position
        })
    else:
        checkpoint_mask = pygame.mask.from_surface(checkpoint_image)
        checkpoints.append({
            'image': checkpoint_image,
            'mask': checkpoint_mask,
            'position': position
        })
        
clock = pygame.time.Clock()
images = [(GRASS, (0, 0)), (TRACK, (0, 0)), (FINISH, FINISH_POSITION)]
WIDTH, HEIGHT = 800,800
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Genetic Algorithm on cars! (Assignment #3)")
    
game_info = GameInfo()
max_time = 50

max_for_this_gen = 0

new_cars = list()
for i in range(len(loaded_list)):
    car = AgentCar()
    car.chromosome = loaded_list[i]
    new_cars.append(car)

active_cars = len(new_cars)



while game_info.get_level_time() < max_time and active_cars > 0:


    if game_info.started == False:
        game_info.start_level()

    draw(WIN, images, new_cars, max_time - game_info.get_level_time(),checkpoints,active_cars, "Test", highest_checkpoint_num, "")

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            break

    active_cars = 0
    for car in new_cars:
        if car.active:
            active_cars+=1
            car.sensorControl()
            car.move_car()
            handle_collision(car, checkpoints, FINISH_MASK)
            highest_checkpoint_num = max(highest_checkpoint_num, car.checkpoints)
            
pygame.quit()  